In [ ]:
# default_exp auth

# auth

> python sdk for interacting with aws backend

In [ ]:
#hide
from nbdev.showdoc import *

In [ ]:
#export
from nicHelper.wrappers import add_method
from lambdasdk.lambdasdk import Lambda
from awsSchema.apigateway import Event, Response
from nicHelper.dictUtil import printDict
from nicHelper.exception import errorString

In [ ]:
#export
class AuthSdk:
  def __init__(
    self,
    user = None,
    pw = None,
    sessionToken = None,
    region = 'ap-southeast-1'
    ):
    self.lambda_ = Lambda(user = user, pw = pw, sessionToken= sessionToken, region = region)
    
  def generalInvoke(self, body={}, functionName= ''):
    event = Event.getInput(body=body)
    response = self.lambda_.invoke(functionName=functionName, input=event)
    try: return Response.parseBody(response)
    except: return response
  

In [ ]:
#hide
sdk = AuthSdk()
# sdk.lambda_.invoke(functionName = 'test', input = {'test':'test'})

In [ ]:
#export
@add_method(AuthSdk)
def auth(self,user,pw,functionName= 'villaAuth-Auth-13UCAIBB6FOHA'):
  return self.generalInvoke(body={'user':user,'password':pw},functionName=functionName)

In [ ]:
result = AuthSdk().auth(user='nic1',pw='12345678')
printDict(result)

AccessKeyId : ASIAVX4Z5T
SecretKey : Xf7MfDSNNU
SessionToken : IQoJb3JpZ2
Expiration : 1605782649.0
AccessToken : eyJraWQiOi
ExpiresIn : 3600
TokenType : Bearer
RefreshToken : eyJjdHkiOi
IdToken : eyJraWQiOi
NewDeviceMetadata
 DeviceKey : ap-southea
 DeviceGroupKey : -lt0cpemm


In [ ]:
#export
@add_method(AuthSdk)
def createUser(self,functionName='villaAuth-CreateUser-ECLM1M2B9LPJ', **kwargs):
  return self.generalInvoke(body=kwargs,functionName=functionName)

In [ ]:
result=AuthSdk().createUser(user='nic6',phone='+66828773682',pw='12345678',name='nic3')
printDict(result)

error : Traceback 
